In [1]:
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import datetime
import math
import random
import subprocess
import time
from collections import OrderedDict
from pathlib import Path

import optuna
optuna.logging.set_verbosity(optuna.logging.INFO)

import mlflow
import numpy as np
import matplotlib.pyplot as plt
import psutil

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data as data
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler

import torchvision
from torchvision.datasets.mnist import MNIST
import torchvision.transforms as transforms

from thop import profile
from tqdm.auto import tqdm
from torchsummary import summary

import warnings
warnings.filterwarnings("ignore")

In [3]:
# torch.cuda.memory._record_memory_history()

In [4]:
def increment_path(path, exist_ok=False, sep='', mkdir=True):
    path = Path(path)
    if path.exists() and not exist_ok:
        path, suffix = (path.with_suffix(''), path.suffix) if path.is_file() else (path, '')
        for n in range(2, 9999):
            p = f'{path}{sep}{n}{suffix}'
            if not os.path.exists(p):
                break
        path = Path(p)
    if mkdir:
        path.mkdir(parents=True, exist_ok=True)
    return path

In [5]:
EXPERIMENT_NAME = "exp"
FILE = Path(os.getcwd()).resolve()
SAVE_DIR = increment_path(FILE / "artifacts" / f"{EXPERIMENT_NAME}", exist_ok=False)
(SAVE_DIR / 'checkpoints').mkdir(parents=True, exist_ok=True)

In [6]:
local_tracking_uri = f"sqlite:///mlflow/mlflow.db"
mlflow.set_tracking_uri(local_tracking_uri)
mlflow.set_experiment(SAVE_DIR.stem)

2024/03/10 22:30:04 INFO mlflow.tracking.fluent: Experiment with name 'exp11' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/Hp/Desktop/mldev/2024/Git/mldevworld/Deep-Learning-Essentials/mlruns/11', creation_time=1710090004074, experiment_id='11', last_update_time=1710090004074, lifecycle_stage='active', name='exp11', tags={}>

In [7]:
def set_experiment_seed(seed=42):
    """
    Set random seeds and CUDA-related flags for experiment reproducibility.
    """
    # Set random seeds for reproducibility
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    # If using GPU, set random seed for CUDA operations
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False  # Set to False for reproducibility

    # Optionally log seed information
    print(f"Random seed set to {seed}")

set_experiment_seed()

Random seed set to 42


In [8]:
# Download MNIST dataset
data_train = MNIST(
    "./data/mnist", download=True, 
    transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.GaussianBlur(kernel_size=1),
        transforms.ToTensor()
    ])
)
        
data_test = MNIST(
    "./data/mnist", download=True, train=False,
    transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ])
)

In [9]:
# Define hyperparameters
N_CLASSES = 10
# EPOCHS = 5
# BATCH_SIZE = 512
# DROPOUT_PROB = 0.2
# USE_BATCHNORM = True
# LABEL_SMOOTHING = 0.1
# LR = 1.34E-03
# WARMUP_PROPORTION = 0.1
# WEIGHT_DECAY = 1e-4
# ACCUMULATION_STEPS = 2

In [10]:
# plt.figure(figsize=(10,10))
# for i in range(25):
#     plt.subplot(5,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(data_train[i][0].permute(1,2,0).numpy(), cmap=plt.cm.binary)
#     plt.xlabel(data_train[i][1])
# plt.show()

`torch.utils.data.DataLoader` supports asynchronous data loading and data augmentation in separate worker subprocesses. The default setting for DataLoader is `num_workers=0`, which means that the data loading is synchronous and done in the main process. As a result the main training process has to wait for the data to be available to continue the execution.

Settin`g num_workers >` 0 enables asynchronous data loading and overlap between the training and data loading. num_workers should be tuned depending on the workload, CPU, GPU, and location of training data.`

DataLo`ader acce`pts pin_me`mory argument, which defaults` to F`alse. When using a GPU it’s better to `set pin_memory=`True, this instru`cts DataLo`ader to use pinned memory and enables faster and asynchronous memory copy from the host to the GPU.

In [11]:
def calculate_num_workers():
    # Calculate the number of CPU cores
    num_cpu_cores = os.cpu_count()

    # Set a safe maximum multiplier value (e.g., 0.5) to avoid using all available resources
    max_multiplier = 0.5

    # Calculate the number of workers based on the available resources
    multiplier = min(sum(psutil.cpu_percent(interval=1, percpu=True)) / 100.0, max_multiplier)
    
    # Ensure that num_workers is at least 1
    num_workers = max(1, int(num_cpu_cores * multiplier))

    return num_workers

In [12]:
def get_dataloaders(batch_size):
    # Define train dataloader
    train_dataloader = DataLoader(
        data_train, batch_size=batch_size, 
        sampler=RandomSampler(data_train), 
        pin_memory=True, num_workers=calculate_num_workers()
    )
    
    # Define test dataloader
    test_dataloader = DataLoader(
        data_test, batch_size=batch_size, 
        sampler=SequentialSampler(data_test),
        pin_memory=True, num_workers=calculate_num_workers()
    )

    return train_dataloader, test_dataloader

# train_dataloader, test_dataloader = get_dataloaders(BATCH_SIZE)

`torch.nn.Conv2d()` has `bias` parameter which defaults to `True` (the same is true for `Conv1d` and `Conv3d` ).

If a `nn.Conv2d` layer is directly followed by a `nn.BatchNorm2d` layer, then the bias in the convolution is not needed, instead use `nn.Conv2d(..., bias=False, ....)`. Bias is not needed because in the first step `BatchNorm` subtracts the mean, which effectively cancels out the effect of bias.

This is also applicable to 1d and 3d convolutions as long as `BatchNorm`(or other normalization layer) normalizes on the same dimension as convolution’s bias.

In [13]:
class LeNet(nn.Module):
    def __init__(self, use_batchnorm, dropout_prob):
        super(LeNet, self).__init__()

        self.use_batchnorm = use_batchnorm
                
        self.features1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=(5, 5), bias=not use_batchnorm),
            self._get_norm_layer(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Dropout2d(p=dropout_prob)
        )
        
        self.features2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=(5, 5), bias=not use_batchnorm),
            self._get_norm_layer(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Dropout2d(p=dropout_prob)
        )

        self.features3 = nn.Sequential(
            nn.Conv2d(16, 120, kernel_size=(5, 5), bias=not use_batchnorm),
            self._get_norm_layer(120),
            nn.ReLU(),
            nn.Dropout2d(p=dropout_prob)
        )

        self.classifier = nn.Sequential(
            nn.Linear(120, 84, bias=True),
            nn.ReLU(),
            nn.Dropout(p=dropout_prob),
            nn.Linear(84, 10, bias=True)
        )

        # Initialize layers
        self.apply(self._initialize_weights)

    def _get_norm_layer(self, channels):
        if self.use_batchnorm:
            return nn.BatchNorm2d(channels)
        else:
            return nn.Identity()

    def _initialize_weights(self, m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            init.kaiming_uniform_(m.weight)
            if m.bias is not None:
                init.constant_(m.bias, 0)
        
    def forward(self, x):
        x = self.features1(x)
        x = self.features2(x)
        x = self.features3(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
def get_model(use_batchnorm, dropout_prob):
    # Instantiate the model
    model = LeNet(use_batchnorm, dropout_prob)
    model = model.to(device)
    
    # Test with a random input
    # summary(model, input_size=(1, 32, 32), batch_size=BATCH_SIZE)
    return model

# model = get_model(USE_BATCHNORM, DROPOUT_PROB)

#### 1. Cross Entropy Loss
The standard cross-entropy loss for classification tasks is given by:

$$ \text{Traditional Cross Entropy Loss: } H(y, \hat{y}) = - \sum_i y_i \log(\hat{y}_i) $$

 - $y_i$ is a binary indicator of whether class $i$ is the correct classification.  
 - $p_i$ is the predicted probability of class $i$.

#### 2. Label Smoothed Cross Entropy
Label Smoothing Cross Entropy Loss introduces a modification to the target distribution:

 $$ \text{Label Smoothed Cross Entropy Loss} = - \sum_i \left( (1 - \text{smoothing}) \cdot 1_{\{y_i\}} + \frac{\text{smoothing}}{C-1} \cdot 1_{\{1 - y_i\}} \right) \cdot \log(p_i) $$

Where:
- $1_{\{y_i\}}$ is a binary indicator of whether class $i$ is the correct classification.
- $p_i$ is the predicted probability of class $i$.
- $C$ is the number of classes.
- $\text{smoothing}$  is the smoothing factor.



In [16]:
class LabelSmoothedCrossEntropy(nn.Module):
    def __init__(self, num_classes, smoothing):
        super(LabelSmoothedCrossEntropy, self).__init__()
        self.num_classes = num_classes
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing

    def forward(self, input_logits, target):
        log_probs = F.log_softmax(input_logits, dim=-1)
        true_dist = torch.zeros_like(log_probs)
        true_dist.fill_(self.smoothing / (self.num_classes - 1))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        loss = -torch.sum(true_dist * log_probs) / input_logits.size(0)
        return loss

In [17]:
def get_loss(label_smoothing):
    # Initialize loss
    # criterion = nn.CrossEntropyLoss()
    return LabelSmoothedCrossEntropy(num_classes=N_CLASSES, smoothing=label_smoothing)

# criterion = get_loss(label_smoothing)

#### Adam w/ Weight Decay

In [18]:
def get_optimizer(model, lr, weight_decay):
    # Initialize optimizer
    # optimizer = optim.Adam(model.parameters(), lr=LR)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    return optimizer

# optimizer =  get_optimizer(LR, WEIGHT_DECAY)

#### Cosine Annealing With Warmup

In [19]:
def get_lr_lambda(initial_lr, warmup_steps, total_steps):
    def lr_lambda(current_step):
        if current_step < warmup_steps:
            return initial_lr + (1.0 - initial_lr) * float(current_step) / float(max(1, warmup_steps))
        else:
            return max(0.0, 0.5 * (1.0 + math.cos(math.pi * (current_step - warmup_steps) / float(total_steps - warmup_steps))))
    return lr_lambda

In [20]:
def get_cosine_with_warmup_scheduler(**kwargs):
    train_dataloader = kwargs['train_dataloader']
    optimizer = kwargs['optimizer']
    
    len_dataloader = len(train_dataloader)
    num_update_steps_per_epoch = max(len_dataloader // kwargs['accumulation_steps'], 1) 
    num_examples = len(train_dataloader.dataset)
    max_steps = math.ceil(kwargs['epochs'] * num_update_steps_per_epoch)
    num_warmup_steps = math.ceil(max_steps * kwargs['warmup_proportion'])
    lr_lambda = get_lr_lambda(kwargs['lr'], num_warmup_steps, max_steps)
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)
    return scheduler

# scheduler = get_cosine_with_warmup_scheduler()

#### Adaptive Gradient Clipping

In [21]:
def unitwise_norm(x, norm_type=2.0):
    if x.ndim <= 1:
        return x.norm(norm_type)
    else:
        return x.norm(norm_type, dim=tuple(range(1, x.ndim)), keepdim=True)


def adaptive_clip_grad(parameters, clip_factor=0.01, eps=1e-3, norm_type=2.0):
    if isinstance(parameters, torch.Tensor):
        parameters = [parameters]
    for p in parameters:
        if p.grad is None:
            continue
        p_data = p.detach()
        g_data = p.grad.detach()
        max_norm = unitwise_norm(p_data, norm_type=norm_type).clamp_(min=eps).mul_(clip_factor)
        grad_norm = unitwise_norm(g_data, norm_type=norm_type)
        clipped_grad = g_data * (max_norm / grad_norm.clamp(min=1e-6))
        new_grads = torch.where(grad_norm < max_norm, g_data, clipped_grad)
        p.grad.detach().copy_(new_grads)

#### Logging Model Parameters

In [22]:
def calculate_flops(model, input_size):
    input_tensor = torch.randn(*input_size).to(device)
    flops, params = profile(model, inputs=(input_tensor,))
    return flops

def calculate_disk_size(model):
    # Calculate model disk size in bytes
    disk_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
    # Convert bytes to megabytes
    disk_size_mb = disk_size_bytes / (1024 * 1024)
    return disk_size_mb

def get_model_metrics(model, train_dataloader):
    num_params = sum(p.numel() for p in model.parameters())
    num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    # Set an example input size (adjust based on your actual input size)
    input_size = tuple(next(iter(train_dataloader))[0][0].unsqueeze(0).shape)

    flops = calculate_flops(model, input_size)
    disk_size = calculate_disk_size(model)

    return {
        "num_params": num_params,
        "num_trainable_params": num_trainable_params,
        "flops": flops,
        "disk_size": disk_size
    }

#### Logging GPU Usage

In [23]:
def get_gpu_usage():
    try:
        # Run nvidia-smi command to get GPU utilization information
        result = subprocess.run(["nvidia-smi", "--query-gpu=utilization.gpu", "--format=csv,nounits,noheader"], stdout=subprocess.PIPE, text=True)
        gpu_usage = int(result.stdout.strip())
        return gpu_usage
    except Exception as e:
        print(f"Error getting GPU usage: {e}")
        return None

#### Train

##### Gradient Penalty
Its primary purpose is to encourage smoothness in the learned representations of the model by penalizing sharp changes or oscillations in the model's gradients.

##### Gradient Accumulation
Gradient accumulation adds gradients over an effective batch of size `batch_per_iter * iters_to_accumulate` (`* num_procs` if distributed). The scale should be calibrated for the effective batch, which means inf/NaN checking, step skipping if inf/NaN grads are found, and scale updates should occur at effective-batch granularity. Also, grads should remain scaled, and the scale factor should remain constant, while grads for a given effective batch are accumulated. If grads are unscaled (or the scale factor changes) before accumulation is complete, the next backward pass will add scaled grads to unscaled grads (or grads scaled by a different factor) after which it’s impossible to recover the accumulated unscaled grads `step` must apply.

##### Automatic Mixed Precision Training
Automatic Mixed Precision (AMP) training is a technique used in deep learning to accelerate training by using a combination of lower-precision and higher-precision numerical representations. The primary idea behind AMP is to use lower-precision data types (such as float16) for some parts of the model computation, while maintaining higher precision (such as float32) for critical numerical stability aspects.

In [24]:
def train(epoch, train_dataloader, model, criterion, optimizer, scheduler, accumulation_steps):
    model.train()
    total_loss = 0.0
    
    # Create a GradScaler once at the beginning of training.
    scaler = torch.cuda.amp.GradScaler()
    
    optimizer.zero_grad()
    with tqdm(total=len(train_dataloader), desc=f"Epoch {epoch}/Training", unit="batch") as pbar:
        for i, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            # Runs the forward pass with autocasting.
            with torch.autocast(device_type=device.type, dtype=torch.float16, enabled=True):
                output = model(images)
                loss = criterion(output, labels)

            # # Scales the loss for autograd.grad's backward pass, producing scaled_grad_params
            # scaled_grad_params = torch.autograd.grad(outputs=scaler.scale(loss), inputs=model.parameters(), create_graph=True)
    
            # # Creates unscaled grad_params before computing the penalty. scaled_grad_params are
            # # not owned by any optimizer, so ordinary division is used instead of scaler.unscale_:
            # inv_scale = 1. / scaler.get_scale()
            # grad_params = [p * inv_scale for p in scaled_grad_params]
    
            # # Computes the penalty term and adds it to the loss
            # with torch.autocast(device_type=device.type, dtype=torch.float16, enabled=True):
            #     grad_norm = 0
            #     for grad in grad_params:
            #         grad_norm += grad.pow(2).sum()
            #     grad_norm = grad_norm.sqrt()
            #     loss = loss + grad_norm

            # Scale the loss by accumulation steps
            if accumulation_steps > 1:
                with torch.autocast(device_type=device.type, dtype=torch.float16, enabled=True):
                    loss = loss / accumulation_steps
                
            # Applies scaling to the backward call as usual.
            # Accumulates leaf gradients that are correctly scaled.
            scaler.scale(loss).backward()

            if (i + 1) % accumulation_steps == 0:
                # Unscales the gradients of optimizer's assigned params in-place
                scaler.unscale_(optimizer)
                # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
                adaptive_clip_grad(model.parameters())

                # optimizer's gradients are already unscaled, so scaler.step does not unscale them,
                # although it still skips optimizer.step() if the gradients contain infs or NaNs.
                scaler.step(optimizer)

                mlflow.log_metric("lr", optimizer.param_groups[0]["lr"])
                scheduler.step()

                # Updates the scale for next iteration.
                scaler.update()
                
                for param in model.parameters():
                    param.grad = None
            
            total_loss += loss.item()
            pbar.update(1)
            pbar.set_postfix(loss=total_loss / (i+1), lr=f"{optimizer.param_groups[0]['lr']:.5f}")

    avg_loss = total_loss / len(train_dataloader.dataset)
    mlflow.log_metric("train_loss", avg_loss)
        
    print(f"[Train][Epoch {epoch}] Average Loss: {avg_loss:.5f}, Updated Learning Rate: {optimizer.param_groups[0]['lr']}")

#### Eval

In [25]:
class BestLossTracker:
    def __init__(self):
        self.best_loss = float('inf')
    
    def update_best_loss(self, avg_loss):
        if avg_loss < self.best_loss:
            self.best_loss = avg_loss
            return True
        return False

In [26]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0.0):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def update_best_loss(self, val_loss):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

        return not self.early_stop

In [27]:
def test(epoch, best_loss_tracker, test_dataloader, model, criterion):
    model.eval()
    total_correct = 0
    total_loss = 0.0

    with tqdm(total=len(test_dataloader), desc=f"Epoch {epoch}/Testing", unit="batch") as pbar: 
        with torch.no_grad():
            for i, (images, labels) in enumerate(test_dataloader):
                images, labels = images.to(device), labels.to(device)
                
                output = model(images)                
                loss = criterion(output, labels)

                total_loss += loss.item()
                pbar.update(1)
                pbar.set_postfix(loss=total_loss / (i+1))
                
                pred = output.detach().max(1)[1]
                total_correct += pred.eq(labels.view_as(pred)).sum()
            
    avg_loss = total_loss / len(test_dataloader.dataset)
    accuracy = total_correct / len(test_dataloader.dataset)

    mlflow.log_metric("val_loss", avg_loss)
    mlflow.log_metric("val_accuracy", accuracy)
    
    if best_loss_tracker.update_best_loss(avg_loss):
        checkpoint_name = f"checkpoint_best_epoch_{epoch}.pth"
        checkpoint_path = str(SAVE_DIR / "checkpoints" / checkpoint_name)
        torch.save(model.state_dict(), checkpoint_path)
        print(f"[Test][Epoch {epoch}] New best model found! Saving checkpoint. Loss: {avg_loss:.5f}, Accuracy: {accuracy:.3f}")
    else:
        print(f"[Test][Epoch {epoch}] Loss: {avg_loss:.5f}, Accuracy: {accuracy:.3f}")
    
    return avg_loss

#### Run

In [28]:
def run(run_name, **kwargs):
    print(kwargs)
    with mlflow.start_run(run_name=run_name):
        mlflow.log_param("torch_version", torch.__version__)
        mlflow.log_param("mlflow_version", mlflow.__version__)

        hyper_params = {
            "accumulation_steps": kwargs['accumulation_steps'], 
            "batch_size": kwargs['batch_size'], 
            "batchnorm": kwargs['batchnorm'],
            "dropout":kwargs['dropout_prob'],
            "epochs": kwargs['epochs'], 
            "label_smoothing": kwargs['label_smoothing'],
            "start_lr": kwargs['lr'],
            "warmup_proportion": kwargs['warmup_proportion'],
            "weight_decay": kwargs['weight_decay']
        }

        mlflow.log_params(hyper_params)
        mlflow.set_tags({"model": "lenet", "dataset": "mnist"})

        train_dataloader, test_dataloader = get_dataloaders(kwargs['batch_size'])
        model = get_model(kwargs['dropout_prob'], kwargs['batchnorm'])
        criterion = get_loss(kwargs['label_smoothing'])
        optimizer = get_optimizer(model, kwargs['lr'], kwargs['weight_decay'])
        
        scheduler_kwargs = {
            'epochs': kwargs['epochs'],
            'train_dataloader':train_dataloader,
            'optimizer': optimizer,
            'lr':kwargs['lr'],
            'accumulation_steps': kwargs['accumulation_steps'],
            'warmup_proportion': kwargs['warmup_proportion']
        }
        scheduler = get_cosine_with_warmup_scheduler(**scheduler_kwargs)
        
        mlflow.pytorch.log_model(model, "model")
        mlflow.log_param("start_time", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

        model_metrics = get_model_metrics(model, train_dataloader)
        mlflow.log_param("num_params", model_metrics['num_params'])
        mlflow.log_param("num_trainable_params", model_metrics['num_trainable_params'])
        mlflow.log_param("flops", model_metrics['flops'])
        mlflow.log_param("disk_size", model_metrics['disk_size'])

        best_loss_tracker = BestLossTracker()
        early_stopping = EarlyStopping(patience=1, delta=1e-6)

        start_time = time.time()
        for epoch in range(kwargs['epochs']):
            train(epoch, train_dataloader, model, criterion, optimizer, scheduler, kwargs['accumulation_steps'])
            val_loss = test(epoch, best_loss_tracker, test_dataloader, model, criterion)
            
            if get_gpu_usage() is not None:
                mlflow.log_metric("gpu", get_gpu_usage(), step=epoch)

            if not early_stopping.update_best_loss(val_loss):
                print("Early stopping triggered.")
                break


        total_time = time.time() - start_time
        avg_epoch_time = total_time / kwargs['epochs']
        avg_batch_time = avg_epoch_time / len(train_dataloader)

        mlflow.log_metric("total_training_time", total_time)
        mlflow.log_metric("average_epoch_time", avg_epoch_time)
        mlflow.log_metric("average_batch_time", avg_batch_time)
        mlflow.log_param("end_time", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    # torch.cuda.memory._dump_snapshot("snapshot.pickle")
    return val_loss

In [29]:
def objective(trial):
    EPOCHS = 5
    accumulation_steps = trial.suggest_int('accumulation_steps', 1, 3)  # Accumulation steps
    batchnorm = trial.suggest_categorical('batchnorm', [True, False])  # Use batch normalization
    batch_size = trial.suggest_int('batch_size', 256, 2048, log=True)  # Batch size
    dropout_prob = trial.suggest_float('dropout_prob', 0.0, 0.4)  # Dropout probability
    label_smoothing = trial.suggest_float('label_smoothing', 0.0, 0.25)  # Label smoothing
    lr = trial.suggest_float('lr', 1e-3, 1e-1, log=True)  # Learning rate
    warmup_proportion = trial.suggest_float('warmup_proportion', 0.0, 0.1)  # Warmup proportion
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)  # Weight decay

    run_name = f"run_{trial.number}"  # Create a unique run_name based on the trial number
    
    # Update the model and optimizer with the sampled hyperparameters
    kwargs = {
        'epochs': EPOCHS,
        'accumulation_steps': accumulation_steps,
        'batchnorm': batchnorm,
        'batch_size': batch_size,
        'dropout_prob': dropout_prob,
        'label_smoothing': label_smoothing,
        'lr': lr,
        'warmup_proportion': warmup_proportion,
        'weight_decay': weight_decay
    }
    
    # Train and evaluate the model
    val_loss = run(run_name=run_name, **kwargs)

    # Return the validation loss for optimization
    return val_loss

In [30]:
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())  # Change 'minimize' to 'maximize' if you're maximizing a metric
study.optimize(objective, n_trials=100)  # Adjust the number of trials as needed

[I 2024-03-10 22:30:04,648] A new study created in memory with name: no-name-105c6438-58f0-4288-beb6-8d0e3d0c0428


{'epochs': 5, 'accumulation_steps': 1, 'batchnorm': True, 'batch_size': 1221, 'dropout_prob': 0.1451223008122237, 'label_smoothing': 0.2069263707464151, 'lr': 0.0014513999543911257, 'warmup_proportion': 0.04213229508497701, 'weight_decay': 0.0006887755240006057}


2024/03/10 22:30:07 WARNING mlflow.utils.requirements_utils: Found torch version (2.2.0+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.2.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/03/10 22:30:11 WARNING mlflow.utils.requirements_utils: Found torch version (2.2.0+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.2.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.


Epoch 0/Training:   0%|          | 0/50 [00:00<?, ?batch/s]

...

Epoch 4/Training:   0%|          | 0/47 [00:00<?, ?batch/s]

[Train][Epoch 4] Average Loss: 0.00026, Updated Learning Rate: 0.0


Epoch 4/Testing:   0%|          | 0/8 [00:00<?, ?batch/s]

[I 2024-03-11 02:40:27,976] Trial 99 finished with value: 0.00023607323169708253 and parameters: {'accumulation_steps': 1, 'batchnorm': False, 'batch_size': 1287, 'dropout_prob': 0.16847003099338972, 'label_smoothing': 0.03425209877627631, 'lr': 0.05775113013921722, 'warmup_proportion': 0.0540230239045516, 'weight_decay': 1.5959801171624955e-06}. Best is trial 84 with value: 4.6606420353055e-05.


[Test][Epoch 4] New best model found! Saving checkpoint. Loss: 0.00024, Accuracy: 0.980


In [31]:
# Print the best hyperparameters and the corresponding value
print('Best trial:')
trial = study.best_trial

print('Value: {}'.format(trial.value))
print('Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Best trial:
Value: 4.6606420353055e-05
Params: 
    accumulation_steps: 1
    batchnorm: False
    batch_size: 1609
    dropout_prob: 0.12931721102481436
    label_smoothing: 8.83890730090731e-07
    lr: 0.028147621640854794
    warmup_proportion: 0.04584907346713653
    weight_decay: 2.1415275778699216e-06
